# Pokemon Battle Environment Tutorial

This notebook demonstrates how to use the OpenEnv Pokemon Battle Environment for reinforcement learning and battle agents.

## Prerequisites

Before running this notebook, ensure:
1. Pokemon Showdown server is running on `localhost:8000`
2. OpenEnv FastAPI server is running on `localhost:9980`

Start the servers with:
```bash
# Terminal 1: Pokemon Showdown
cd pokemon-showdown
node pokemon-showdown start --no-security

# Terminal 2: OpenEnv Server
cd OpenEnv
$env:PYTHONPATH='src'
python -m uvicorn envs.pokemon_env.server.app:app --host 127.0.0.1 --port 9980
```

## 1. Basic Setup and Connection

First, let's import the necessary modules and connect to the environment.

In [18]:
import sys
import os

# Add OpenEnv to path
OPENENV_PATH = os.path.join(os.getcwd(), 'OpenEnv', 'src')
if OPENENV_PATH not in sys.path:
    sys.path.insert(0, OPENENV_PATH)

from envs.pokemon_env import PokemonEnv, PokemonAction

# Connect to the environment
BASE_URL = "http://localhost:9980"
env = PokemonEnv(base_url=BASE_URL)

print(f"✓ Connected to Pokemon Battle Environment at {BASE_URL}")

✓ Connected to Pokemon Battle Environment at http://localhost:9980


## 2. Starting a Battle with reset()

The `reset()` method starts a fresh battle and returns the initial observation.

In [19]:
# Start a new battle
result = env.reset()
observation = result.observation

print(f"Battle Format: {observation.battle_format}")
print(f"Battle ID: {observation.battle_id}")
print(f"Turn: {observation.turn}")
print(f"Done: {result.done}")
print(f"Reward: {result.reward}")

Battle Format: gen8randombattle
Battle ID: battle-gen8randombattle-1
Turn: 1
Done: False
Reward: 0.0


## 3. Understanding the Observation

Each observation contains comprehensive battle state information.

In [20]:
# View your active Pokemon
if observation.active_pokemon:
    active = observation.active_pokemon
    print("Your Active Pokemon:")
    print(f"  Species: {active.species}")
    print(f"  HP: {active.current_hp}/{active.max_hp} ({active.hp_percent:.1%})")
    print(f"  Level: {active.level}")
    print(f"  Types: {active.types}")
    print(f"  Ability: {active.ability}")
    print(f"  Item: {active.item}")
    print(f"  Status: {active.status or 'None'}")
    print(f"  Fainted: {active.fainted}")
    print()
    print("  Base Stats:")
    print(f"    ATK: {active.attack}, DEF: {active.defense}")
    print(f"    SpA: {active.special_attack}, SpD: {active.special_defense}")
    print(f"    SPE: {active.speed}")
    print()
    print(f"  Stat Boosts: {active.boosts}")

Your Active Pokemon:
  Species: ferrothorn
  HP: 241/241 (100.0%)
  Level: 77
  Types: ['GRASS', 'STEEL']
  Ability: ironbarbs
  Item: leftovers
  Status: None
  Fainted: False

  Base Stats:
    ATK: 94, DEF: 131
    SpA: 54, SpD: 116
    SPE: 20

  Stat Boosts: {'accuracy': 0, 'atk': 0, 'def': 0, 'evasion': 0, 'spa': 0, 'spd': 0, 'spe': 0}


In [21]:
# View opponent's active Pokemon
if observation.opponent_active_pokemon:
    opp = observation.opponent_active_pokemon
    print("Opponent's Active Pokemon:")
    print(f"  Species: {opp.species}")
    print(f"  HP: {opp.current_hp}/{opp.max_hp} ({opp.hp_percent:.1%})")
    print(f"  Level: {opp.level}")
    print(f"  Types: {opp.types}")
    print(f"  Ability: {opp.ability}")
    print(f"  Item: {opp.item}")
    print(f"  Status: {opp.status or 'None'}")

Opponent's Active Pokemon:
  Species: nidoqueen
  HP: 100/100 (100.0%)
  Level: 82
  Types: ['POISON', 'GROUND']
  Ability: None
  Item: unknown_item
  Status: None


## 4. Available Moves and Their Details

The observation includes detailed information about available moves.

In [22]:
# Display available moves
print(f"Available Move Indices: {observation.available_moves}")
print()

if observation.active_pokemon and observation.active_pokemon.moves:
    print("Move Details:")
    for idx in observation.available_moves:
        if 0 <= idx < len(observation.active_pokemon.moves):
            move = observation.active_pokemon.moves[idx]
            print(f"  [{idx}] {move['id'].upper()}")
            print(f"      Type: {move['type']}")
            print(f"      Power: {move['power']}")
            print(f"      PP: {move['pp']}")
            print(f"      Accuracy: {move['accuracy']*100:.0f}%")
            print()

Available Move Indices: [0, 1, 2, 3]

Move Details:
  [0] PROTECT
      Type: NORMAL (pokemon type) object
      Power: 0
      PP: 16
      Accuracy: 100%

  [1] POWERWHIP
      Type: GRASS (pokemon type) object
      Power: 120
      PP: 16
      Accuracy: 85%

  [2] GYROBALL
      Type: STEEL (pokemon type) object
      Power: 0
      PP: 8
      Accuracy: 100%

  [3] LEECHSEED
      Type: GRASS (pokemon type) object
      Power: 0
      PP: 16
      Accuracy: 90%



## 5. Your Full Team

View all Pokemon on your team, including those on the bench.

In [23]:
# Display your team
print(f"Available Switch Indices: {observation.available_switches}")
print()
print("Your Full Team:")
for i, pokemon in enumerate(observation.team):
    if pokemon:
        status_flags = []
        if pokemon.active:
            status_flags.append("ACTIVE")
        if pokemon.fainted:
            status_flags.append("FAINTED")
        if i in observation.available_switches:
            status_flags.append("can switch")
        
        status_str = f" [{', '.join(status_flags)}]" if status_flags else ""
        print(f"  [{i}] {pokemon.species}: {pokemon.current_hp}/{pokemon.max_hp} HP{status_str}")

Available Switch Indices: [0, 1, 2, 3, 4]

Your Full Team:
  [0] ferrothorn: 241/241 HP [ACTIVE, can switch]
  [1] gothitelle: 266/266 HP [can switch]
  [2] blissey: 572/572 HP [can switch]
  [3] wailord: 457/457 HP [can switch]
  [4] aerodactyl: 265/265 HP [can switch]
  [5] marshadow: 235/235 HP


## 6. Field Conditions and Battle State

Track weather, terrain, and other field effects.

In [24]:
# Display field conditions
print("Field Conditions:")
print(f"  Weather: {observation.field_conditions.get('weather', 'None')}")
print(f"  Terrain: {observation.field_conditions.get('terrain', 'None')}")
print(f"  Trick Room: {observation.field_conditions.get('trick_room', False)}")
print()

# Side conditions
your_conditions = observation.field_conditions.get('side_conditions', {})
opp_conditions = observation.field_conditions.get('opponent_side_conditions', {})

if your_conditions:
    print(f"  Your Side Conditions: {your_conditions}")
if opp_conditions:
    print(f"  Opponent Side Conditions: {opp_conditions}")

Field Conditions:
  Weather: None
  Terrain: None
  Trick Room: False



## 7. Special Battle Mechanics

Check if special mechanics like Mega Evolution, Dynamax, or Terastallization are available.

In [ ]:
# Special mechanics availability
print("Special Mechanics Available:")
print(f"  Can Mega Evolve: {observation.can_mega_evolve}")
print(f"  Can Dynamax: {observation.can_dynamax}")
print(f"  Can Terastallize: {observation.can_terastallize}")
print(f"  Forced Switch: {observation.forced_switch}")

PokemonObservation(done=False, reward=0.0, metadata={}, active_pokemon=PokemonData(species='ferrothorn', hp_percent=1.0, max_hp=241, current_hp=241, level=77, status=None, types=['GRASS', 'STEEL'], ability='ironbarbs', item='leftovers', attack=94, defense=131, special_attack=54, special_defense=116, speed=20, boosts={'accuracy': 0, 'atk': 0, 'def': 0, 'evasion': 0, 'spa': 0, 'spd': 0, 'spe': 0}, moves=[{'id': 'protect', 'type': 'NORMAL (pokemon type) object', 'power': 0, 'pp': 16, 'accuracy': 1.0, 'category': 'STATUS (move category) object'}, {'id': 'powerwhip', 'type': 'GRASS (pokemon type) object', 'power': 120, 'pp': 16, 'accuracy': 0.85, 'category': 'PHYSICAL (move category) object'}, {'id': 'gyroball', 'type': 'STEEL (pokemon type) object', 'power': 0, 'pp': 8, 'accuracy': 1.0, 'category': 'PHYSICAL (move category) object'}, {'id': 'leechseed', 'type': 'GRASS (pokemon type) object', 'power': 0, 'pp': 16, 'accuracy': 0.9, 'category': 'STATUS (move category) object'}], fainted=False

## 8. Taking Actions - Using Moves

Execute a move by creating a `PokemonAction` and passing it to `step()`.

In [16]:
# Choose the first available move
if observation.available_moves and not observation.forced_switch:
    move_idx = observation.available_moves[0]
    action = PokemonAction(
        action_type="move",
        action_index=move_idx
    )
    
    print(f"Using move index {move_idx}")
    if observation.active_pokemon and move_idx < len(observation.active_pokemon.moves):
        move_name = observation.active_pokemon.moves[move_idx]['id']
        print(f"Move name: {move_name}")
    
    # Execute the action
    result = env.step(action)
    new_obs = result.observation
    
    print(f"\nAfter action:")
    print(f"  Turn: {new_obs.turn}")
    print(f"  Done: {result.done}")
    print(f"  Reward: {result.reward}")
    
    if new_obs.active_pokemon:
        print(f"  Your HP: {new_obs.active_pokemon.current_hp}/{new_obs.active_pokemon.max_hp}")
    if new_obs.opponent_active_pokemon:
        print(f"  Opp HP: {new_obs.opponent_active_pokemon.current_hp}/{new_obs.opponent_active_pokemon.max_hp}")

Using move index 0
Move name: outrage

After action:
  Turn: 2
  Done: False
  Reward: None
  Your HP: 234/259
  Opp HP: 53/100


## 9. Taking Actions - Switching Pokemon

Switch to a different Pokemon from your team.

In [17]:
# Get latest observation
observation = result.observation if 'result' in locals() else env.reset().observation

# Switch to a different Pokemon if available
if observation.available_switches:
    switch_idx = observation.available_switches[0]
    target_pokemon = observation.team[switch_idx]
    
    action = PokemonAction(
        action_type="switch",
        action_index=switch_idx
    )
    
    print(f"Switching to: {target_pokemon.species} (index {switch_idx})")
    
    result = env.step(action)
    new_obs = result.observation
    
    print(f"\nAfter switch:")
    if new_obs.active_pokemon:
        print(f"  New active: {new_obs.active_pokemon.species}")
        print(f"  HP: {new_obs.active_pokemon.current_hp}/{new_obs.active_pokemon.max_hp}")
else:
    print("No switches available (all other Pokemon fainted or currently active)")

No switches available (all other Pokemon fainted or currently active)


## 10. Using Special Mechanics

Execute moves with Mega Evolution, Dynamax, or Terastallization.

In [18]:
# Get latest observation
observation = result.observation if 'result' in locals() else env.reset().observation

# Example: Use a move with Mega Evolution if available
if observation.can_mega_evolve and observation.available_moves:
    move_idx = observation.available_moves[0]
    action = PokemonAction(
        action_type="move",
        action_index=move_idx,
        mega_evolve=True
    )
    print("Using move with Mega Evolution!")
    result = env.step(action)
elif observation.can_dynamax and observation.available_moves:
    move_idx = observation.available_moves[0]
    action = PokemonAction(
        action_type="move",
        action_index=move_idx,
        dynamax=True
    )
    print("Using move with Dynamax!")
    result = env.step(action)
elif observation.can_terastallize and observation.available_moves:
    move_idx = observation.available_moves[0]
    action = PokemonAction(
        action_type="move",
        action_index=move_idx,
        terastallize=True
    )
    print("Using move with Terastallization!")
    result = env.step(action)
else:
    print("No special mechanics available this turn")

No special mechanics available this turn


## 11. Battle Metadata and State

Access metadata about the battle, including forfeit status and request IDs.

In [19]:
# Get latest observation
observation = result.observation if 'result' in locals() else env.reset().observation

# Display metadata
print("Battle Metadata:")
pprint(observation.metadata)

# Check environment state
state = env.state()
print("\nEnvironment State:")
print(f"  Episode ID: {state.episode_id}")
print(f"  Step Count: {state.step_count}")
print(f"  Battle Format: {state.battle_format}")
print(f"  Player Username: {state.player_username}")
print(f"  Battle ID: {state.battle_id}")
print(f"  Battle Finished: {state.is_battle_finished}")
print(f"  Battle Winner: {state.battle_winner}")

Battle Metadata:
{}

Environment State:
  Episode ID: a1247f34-22c5-4711-970c-7a85459462c8
  Step Count: 1
  Battle Format: gen8randombattle
  Player Username: player
  Battle ID: battle-gen8randombattle-1
  Battle Finished: False
  Battle Winner: None


## 12. Running a Complete Battle

Let's run a simple agent that makes random legal moves until the battle ends.

In [20]:
import random

# Start a fresh battle
result = env.reset()
print(f"Starting battle: {result.observation.battle_id}")
print()

max_turns = 50
for turn_num in range(max_turns):
    observation = result.observation
    
    # Check if battle is done
    if result.done:
        print(f"Battle ended on turn {observation.turn}")
        print(f"Reward: {result.reward}")
        if observation.metadata:
            print(f"Metadata: {observation.metadata}")
        break
    
    # Handle forced switch
    if observation.forced_switch and observation.available_switches:
        switch_idx = random.choice(observation.available_switches)
        action = PokemonAction(action_type="switch", action_index=switch_idx)
        print(f"Turn {observation.turn}: Forced switch to index {switch_idx}")
    # Choose random move
    elif observation.available_moves:
        move_idx = random.choice(observation.available_moves)
        action = PokemonAction(action_type="move", action_index=move_idx)
        if observation.active_pokemon and move_idx < len(observation.active_pokemon.moves):
            move_name = observation.active_pokemon.moves[move_idx]['id']
            print(f"Turn {observation.turn}: Using {move_name}")
    # No legal actions available, wait
    else:
        print(f"Turn {observation.turn}: No legal actions, waiting...")
        continue
    
    # Execute action
    result = env.step(action)
else:
    print(f"\nReached {max_turns} turn limit")

Starting battle: battle-gen8randombattle-1

Turn 2: Using outrage
Turn 3: Using outrage
Turn 4: Using firepunch
Turn 5: Using firepunch
Turn 5: Forced switch to index 2
Turn 6: Using sparklingaria
Turn 7: Using toxic
Turn 8: Using toxic
Turn 8: Forced switch to index 0
Turn 9: Using leechseed
Turn 10: Using protect
Turn 11: Using gyroball
Turn 12: Using gyroball
Turn 13: Using gyroball
Turn 14: Using protect
Turn 15: Using leechseed
Turn 16: Using leechseed
Turn 17: Using stealthrock
Turn 18: Using stealthrock
Turn 19: Using gyroball
Turn 20: Using leechseed
Turn 21: Using protect
Turn 22: Using gyroball
Turn 23: Using leechseed
Turn 24: Using gyroball
Turn 25: Using protect
Turn 26: Using stealthrock
Turn 27: Using leechseed
Turn 28: Using protect
Turn 29: Using protect
Turn 30: Using protect
Turn 31: Using gyroball
Turn 32: Using protect
Turn 33: Using gyroball
Turn 33: Forced switch to index 1
Turn 34: Using rest
Turn 35: Using toxic
Battle ended on turn 35
Reward: 1.0


## 13. Building a Simple Strategy Agent

Example of an agent that prioritizes high-power moves and defensive switches.

In [21]:
def choose_best_move(observation):
    """Simple strategy: choose highest power move, or switch if low HP."""
    
    # If low HP and can switch, do so
    if observation.active_pokemon:
        hp_percent = observation.active_pokemon.hp_percent
        if hp_percent < 0.3 and observation.available_switches:
            # Find healthiest switch option
            best_switch = None
            best_hp = 0
            for switch_idx in observation.available_switches:
                pokemon = observation.team[switch_idx]
                if pokemon and pokemon.hp_percent > best_hp:
                    best_hp = pokemon.hp_percent
                    best_switch = switch_idx
            
            if best_switch is not None:
                return PokemonAction(action_type="switch", action_index=best_switch)
    
    # Otherwise, choose highest power move
    if observation.available_moves and observation.active_pokemon:
        best_move_idx = None
        best_power = -1
        
        for move_idx in observation.available_moves:
            if move_idx < len(observation.active_pokemon.moves):
                move = observation.active_pokemon.moves[move_idx]
                power = move.get('power', 0)
                if power > best_power:
                    best_power = power
                    best_move_idx = move_idx
        
        if best_move_idx is not None:
            return PokemonAction(action_type="move", action_index=best_move_idx)
    
    # Fallback: forced switch
    if observation.forced_switch and observation.available_switches:
        return PokemonAction(action_type="switch", action_index=observation.available_switches[0])
    
    return None

# Test the strategy
result = env.reset()
print(f"Testing strategy agent on battle: {result.observation.battle_id}\n")

for turn_num in range(50):
    observation = result.observation
    
    if result.done:
        print(f"\nBattle ended on turn {observation.turn}")
        print(f"Final reward: {result.reward}")
        break
    
    action = choose_best_move(observation)
    if action is None:
        print(f"Turn {observation.turn}: No action available")
        continue
    
    action_desc = f"{action.action_type} #{action.action_index}"
    print(f"Turn {observation.turn}: {action_desc}")
    
    result = env.step(action)

Testing strategy agent on battle: battle-gen8randombattle-2

Turn 1: move #0
Turn 2: move #0
Turn 3: move #0
Turn 4: move #0
Turn 5: move #0
Turn 6: move #0
Turn 7: move #0
Turn 8: move #0
Turn 9: switch #1
Turn 10: move #3
Turn 11: move #3
Turn 12: move #3
Turn 13: move #3
Turn 14: move #3
Turn 15: move #3
Turn 16: move #3
Turn 17: move #3
Turn 18: move #3
Turn 19: move #3
Turn 20: move #3
Turn 21: move #3
Turn 22: move #3
Turn 23: move #3
Turn 24: move #3
Turn 25: move #3
Turn 26: move #2
Turn 27: move #2
Turn 28: move #2
Turn 29: move #2
Turn 30: move #2
Turn 31: move #2
Turn 32: move #2
Turn 33: move #2
Turn 34: move #2
Turn 35: move #2
Turn 36: move #2
Turn 37: move #2
Turn 38: move #2
Turn 39: move #2
Turn 40: move #2
Turn 41: move #2
Turn 42: move #2
Turn 43: move #2
Turn 44: move #2
Turn 45: move #2
Turn 46: move #2
Turn 47: move #2
Turn 48: move #2
Turn 49: move #2
Turn 50: move #0


## 14. Cleanup

Always close the environment when done to clean up resources.

In [22]:
# Close the environment
env.close()
print("Environment closed successfully")

Environment closed successfully
